<a href="https://colab.research.google.com/github/LazyHooks/Aon_Optimus/blob/main/optimus_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

In [ ]:
from transformers import BertTokenizer
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.nn.utils import clip_grad_norm_
from tqdm.notebook import tqdm
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import Dropout

input

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')


Descriptive

In [ ]:
df.shape
df.info()

In [ ]:
df.head()

In [ ]:
df.isnull().sum()
df.describe()

Split X & Y

In [ ]:
X = dataset.iloc[:, 1:-1].values
Y = dataset.iloc[:, -1].values

Scaling ?

Tokenizing

In [ ]:
#PRETRAINED_LM = "bert-base-uncased"
#tokenizer = BertTokenizer.from_pretrained(PRETRAINED_LM, do_lower_case=True)
#tokenizer

In [1]:
"""
def encode(docs):
    '''
    This function takes list of texts and returns input_ids and attention_mask of texts
    '''
    encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=128, padding='max_length',
                            return_attention_mask=True, truncation=True, return_tensors='pt')
    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    return input_ids, attention_masks
    """

"\ndef encode(docs):\n    '''\n    This function takes list of texts and returns input_ids and attention_mask of texts\n    '''\n    encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=128, padding='max_length',\n                            return_attention_mask=True, truncation=True, return_tensors='pt')\n    input_ids = encoded_dict['input_ids']\n    attention_masks = encoded_dict['attention_mask']\n    return input_ids, attention_masks\n    "

Build Classifier

In [ ]:
"""
from transformers import BertForSequenceClassification
N_labels = len(train_df.label.unique())
model = BertForSequenceClassification.from_pretrained(PRETRAINED_LM,
                                                      num_labels=N_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
"""

Build Function

In [ ]:
def build_classifier(optimizer):
    #initialise ANN
    classifier = Sequential()

    #layers w/ dropout
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation ='relu',input_dim=11))
    classifier.add(Dropout(p=0.1))
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation ='relu'))
    classifier.add(Dropout(p=0.1))

    #output layer
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation ='sigmoid'))

    #compile
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

Tuning

In [ ]:
EPOCHS = 30
LEARNING_RATE = 2e-6

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, 
             num_warmup_steps=0,
            num_training_steps=len(train_dataloader)*EPOCHS )

In [ ]:
Training

In [ ]:
#init
classifier = KerasClassifier(build_fn = build_classifier)

#search
parameters = {'batch_size':[25,32], 'epochs':[100, 500],'optimizer':['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator=classifier, param_grid=parameters, scoring = 'accuracy', cv=10)


In [ ]:
#fit
grid_search=grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_ 
best_accuracy =grid_search.best_score_

predictions

In [ ]:
# predict
y_pred = classifier.predict(X_test)

# Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)